# Setup

In [1]:
# All import statements needed for the project
import json
import requests
import pandas as pd
import urllib.parse
import requests
import pathlib
from pathlib import Path
import psycopg2
import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely
import sqlalchemy as db

from sqlalchemy.orm import declarative_base

In [2]:
# Any constants you might need; some have been added for you

# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = DATA_DIR / "zipcodes" / "ZIP_CODE_040114.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

NYC_DATA_APP_TOKEN = "qusZQnQU0ua9VbjGpXFDVhsbK"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/"
NYC_DATA_311 = "erm2-nwe9.geojson"
NYC_DATA_TREES = "5rq2-4hqu.geojson"

DB_NAME = "FINAL_PROJECT_DATA"
DB_USER = "sareayang"
DB_URL = f"postgresql+psycopg2://{DB_USER}@localhost/{DB_NAME}"
DB_SCHEMA_FILE = "schema.sql"
# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

In [3]:
# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

# Part 1: Data Preprocessing

In [4]:
def download_nyc_geojson_data(url, file_name, force=False):
    filename = Path(file_name + '.geojson')
    
    if force or not filename.exists():
        print(f"Downloading {url} to {file_name}...")
    
        response = requests.get(url)
        text = response.text

        with open(file_name + '.geojson', 'w') as file:
            file.write(text)
        print(f"Done downloading {url}.")
        
    else:
        print(f"Reading from {file_name}...")
    
    gdf = gpd.read_file(file_name + '.geojson', driver = 'GeoJSON')
    return gdf

In [5]:
def download_and_clean_311_data():
    url = "https://data.cityofnewyork.us/resource/erm2-nwe9.geojson?$$app_token=RbFfvU4T8a7C7rDHaA9eqAkvZ"
    file_name = 'complaints_head(1000)'
    complaints = download_nyc_geojson_data(url, file_name, force=False)
    
    # choose columns we neeed
    new_complaints = complaints[['created_date', 'incident_zip', 'complaint_type', 'geometry']]
    
    # normalize SRID to EPSG:4326
    new_complaints = new_complaints.to_crs(epsg=4326)
    
    # 去除NaN
    columns_with_nan = new_complaints.columns[new_complaints.isnull().any()].tolist()
    
    for column_name in columns_with_nan:
        print("Processing column:", column_name)
        new_complaints = new_complaints.dropna(subset=[column_name])
        print("Removed rows with NaN value in", column_name)

    return new_complaints

In [6]:
def download_and_clean_tree_data():
    url = "https://data.cityofnewyork.us/resource/5rq2-4hqu.geojson?$$app_token=RbFfvU4T8a7C7rDHaA9eqAkvZ"
    file_name = 'trees_head(1000)'
    trees = download_nyc_geojson_data(url, file_name, force=False)
    
    # normalize SRID to EPSG:4326
    trees = trees.to_crs(epsg=4326)
    
    # 去除NaN
    columns_with_nan = trees.columns[trees.isnull().any()].tolist()
    
    for column_name in columns_with_nan:
        print("Processing column:", column_name)
        trees = trees.dropna(subset=[column_name])
        print("Removed rows with NaN value in", column_name)
        
    # choose columns we neeed
    new_trees = trees[['tree_id', 'zipcode', 'spc_common', 'health', 'status', 'latitude', 'longitude', 'geometry']].copy()

    return new_trees

In [7]:
def load_and_clean_zipcodes(zipcode_datafile):
    zipcode_data = gpd.read_file(zipcode_datafile)
    
    zipcodes = zipcode_data[['ZIPCODE', 'geometry']]
    zipcodes = zipcodes.rename(columns={'ZIPCODE': 'zipcode'})
    
    # normalize SRID to EPSG:4326
    zipcodes = zipcodes.to_crs(epsg=4326)
    
    # Convert 'Polygon' column to a format supported by SQL
    # Replace 'polygon_column' with your actual 'Polygon' column name
    ## zipcodes['geometry'] =zipcodes['geometry'].apply(lambda x: x.wkt if x else None)

    # Define data types explicitly (required for 'Polygon' column)
    ## data_types = {'geometry': String}  # Adjust the data type as per your SQL schema
    return zipcodes

In [8]:
def load_and_clean_zillow_data(rent_datafile):
    zillow_rent_data = pd.read_csv(rent_datafile)
    
    columns_to_drop = ['RegionID', 'SizeRank', 'RegionType', 'State', 'StateName', 'City', 'Metro', 'CountyName']
    df_zillow_data = zillow_rent_data.drop(columns=columns_to_drop)
    
    # rename columns
    df_zillow_data = df_zillow_data.rename(columns={'RegionName': 'rent_zip'})
    
    # normalize rent data
    rents = pd.melt(df_zillow_data, id_vars=['rent_zip'], var_name='rent_date', value_name='rent_amount')
    
    # set datetime type
    rents['rent_date'] = pd.to_datetime(rents['rent_date'])
    rents['rent_zip']
    
    return rents

In [9]:
# ZIPCODE_DATA_FILE = DATA_DIR / "zipcodes" / "ZIP_CODE_040114.shp"
ZIPCODE_DATA_FILE = 'data/nyc_zipcodes.shp'
RENT_DATA_FILE = 'data/zillow_rent_data.csv'

In [10]:
def load_all_data():
    geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
    geodf_311_data = download_and_clean_311_data()
    geodf_tree_data = download_and_clean_tree_data()
    df_zillow_data = load_and_clean_zillow_data(RENT_DATA_FILE)
    return (
        geodf_zipcode_data,
        geodf_311_data,
        geodf_tree_data,
        df_zillow_data
    )

In [11]:
geodf_zipcode_data, geodf_311_data, geodf_tree_data, df_zillow_data = load_all_data()

Reading from complaints_head(1000)...
Processing column: incident_zip
Removed rows with NaN value in incident_zip
Processing column: geometry
Removed rows with NaN value in geometry
Reading from trees_head(1000)...
Processing column: health
Removed rows with NaN value in health
Processing column: steward
Removed rows with NaN value in steward
Processing column: spc_common
Removed rows with NaN value in spc_common
Processing column: problems
Removed rows with NaN value in problems
Processing column: spc_latin
Removed rows with NaN value in spc_latin
Processing column: guards
Removed rows with NaN value in guards
Processing column: sidewalk
Removed rows with NaN value in sidewalk


In [12]:
# Show basic info about each dataframe
geodf_zipcode_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   zipcode   263 non-null    object  
 1   geometry  263 non-null    geometry
dtypes: geometry(1), object(1)
memory usage: 4.2+ KB


In [13]:
# Show first 5 entries about each dataframe
geodf_zipcode_data.head()

,zipcode,geometry
0,11436,"POLYGON ((-73.80585 40.68291, -73.80569 40.682..."
1,11213,"POLYGON ((-73.93740 40.67973, -73.93487 40.679..."
2,11212,"POLYGON ((-73.90294 40.67084, -73.90223 40.668..."
3,11225,"POLYGON ((-73.95797 40.67066, -73.95576 40.670..."
4,11218,"POLYGON ((-73.97208 40.65060, -73.97192 40.650..."


In [14]:
geodf_311_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 986 entries, 0 to 999
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   created_date    986 non-null    datetime64[ns]
 1   incident_zip    986 non-null    object        
 2   complaint_type  986 non-null    object        
 3   geometry        986 non-null    geometry      
dtypes: datetime64[ns](1), geometry(1), object(2)
memory usage: 38.5+ KB


In [15]:
geodf_311_data.head()

,created_date,incident_zip,complaint_type,geometry
0,2023-12-06 12:00:00,11357,Derelict Vehicles,POINT (-73.81909 40.77822)
1,2023-12-06 12:00:00,11423,Derelict Vehicles,POINT (-73.77700 40.70893)
2,2023-12-06 01:21:27,10458,Noise - Residential,POINT (-73.89632 40.85988)
3,2023-12-06 01:20:45,11357,Illegal Parking,POINT (-73.81744 40.77804)
4,2023-12-06 01:20:17,11214,Illegal Parking,POINT (-73.98806 40.59708)


In [16]:
geodf_tree_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 973 entries, 0 to 999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   tree_id     973 non-null    object  
 1   zipcode     973 non-null    object  
 2   spc_common  973 non-null    object  
 3   health      973 non-null    object  
 4   status      973 non-null    object  
 5   latitude    973 non-null    object  
 6   longitude   973 non-null    object  
 7   geometry    973 non-null    geometry
dtypes: geometry(1), object(7)
memory usage: 68.4+ KB


In [17]:
geodf_tree_data.head()

,tree_id,zipcode,spc_common,health,status,latitude,longitude,geometry
0,180683,11375,red maple,Fair,Alive,40.72309177,-73.84421522,POINT (-73.84422 40.72309)
1,200540,11357,pin oak,Fair,Alive,40.79411067,-73.81867946,POINT (-73.81868 40.79411)
2,204026,11211,honeylocust,Good,Alive,40.71758074,-73.9366077,POINT (-73.93661 40.71758)
3,204337,11211,honeylocust,Good,Alive,40.71353749,-73.93445616,POINT (-73.93446 40.71354)
4,189565,11215,American linden,Good,Alive,40.66677776,-73.97597938,POINT (-73.97598 40.66678)


In [18]:
df_zillow_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705810 entries, 0 to 705809
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   rent_zip     705810 non-null  int64         
 1   rent_date    705810 non-null  datetime64[ns]
 2   rent_amount  250167 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 16.2 MB


In [19]:
df_zillow_data.head()

,rent_zip,rent_date,rent_amount
0,77494,2015-01-31,1606.206406
1,77449,2015-01-31,1257.814660
2,77084,2015-01-31,NaN
3,79936,2015-01-31,NaN
4,11385,2015-01-31,NaN


# Part 2: Storing Data

In [20]:
def setup_new_postgis_database(username, db_name):
    raise NotImplementedError()

### Creating Tables

In [21]:
engine = db.create_engine(DB_URL)

In [22]:
# if using SQL (as opposed to SQLAlchemy), define the SQL statements to create your 4 tables
ZIPCODE_SCHEMA = """
CREATE TABLE nyc_zip_codes (
    id SERIAL PRIMARY KEY,
    zip_code INTEGER,
    geom POLYGON
)
"""
# geometry_column geometry(GeometryType, SRID)

NYC_311_SCHEMA = """
CREATE TABLE complaints_311 (
    id SERIAL PRIMARY KEY,
    created_date TIMESTAMP,
    incident_zip INTEGER,
    complaints_type TEXT,
    latitude NUMERIC(6),
    longitude NUMERIC(6)
)
"""

NYC_TREE_SCHEMA = """
CREATE TABLE trees (
    id SERIAL PRIMARY KEY,
    tree_id INTEGER,
    tree_zip INTEGER,
    spc_common TEXT,
    health TEXT,
    status TEXT,
    geometry POINT, 
    latitude NUMERIC(6),
    longitude NUMERIC(6)
)
"""

ZILLOW_SCHEMA = """
CREATE TABLE rents (
    id SERIAL PRIMARY KEY,
    rent_zip INTEGER,
    rent_date DATE,
    rent NUMERIC(6)
)
"""

In [23]:
# create that required schema.sql file
with open(DB_SCHEMA_FILE, "w") as f:
    f.write(ZIPCODE_SCHEMA)
    f.write(NYC_311_SCHEMA)
    f.write(NYC_TREE_SCHEMA)
    f.write(ZILLOW_SCHEMA)

In [24]:
# If using SQL (as opposed to SQLAlchemy), execute the schema files to create tables
with engine.connect() as connection:
    
    connection.execute("DROP TABLE IF EXISTS nyc_zip_codes, complaints_311, trees, rents CASCADE") # if table already exists
    
    connection.execute(ZIPCODE_SCHEMA)
    connection.execute(NYC_311_SCHEMA)
    connection.execute(NYC_TREE_SCHEMA)
    connection.execute(ZILLOW_SCHEMA)

### Add Data to Database

In [25]:
# dataframe to sql table
df_zillow_data.to_sql('rents', con=engine, if_exists='replace', index=False)

# GeoDataFrame data into a PostGIS-enabled PostgreSQL table
geodf_311_data.to_postgis('complaints', con=engine, if_exists='replace', index=False)
geodf_zipcode_data.to_postgis('zipcodes', con=engine, if_exists='replace', index=False)
geodf_tree_data.to_postgis('trees', con=engine, if_exists='replace', index=False)

/Users/sareayang/anaconda3/lib/python3.11/site-packages/pandas/io/sql.py:1781: SAWarning: Did not recognize type 'point' of column 'geometry'
  self.meta.reflect(bind=self.connectable, only=[table_name], schema=schema)


In [26]:
### test: check the table
with engine.connect() as connection:
    result = connection.execute(
        """
        SELECT * FROM trees LIMIT 5
        """
    )

for row in result:
    print(row)

('180683', '11375', 'red maple', 'Fair', 'Alive', '40.72309177', '-73.84421522', '0101000020E61000008DB8459F077652C0758170458E5C4440')
('200540', '11357', 'pin oak', 'Fair', 'Alive', '40.79411067', '-73.81867946', '0101000020E6100000F6E0863E657452C0FE45186BA5654440')
('204026', '11211', 'honeylocust', 'Good', 'Alive', '40.71758074', '-73.9366077', '0101000020E61000008D197161F17B52C08B7B89AFD95B4440')
('204337', '11211', 'honeylocust', 'Good', 'Alive', '40.71353749', '-73.93445616', '0101000020E61000001DD33421CE7B52C0A5635632555B4440')
('189565', '11215', 'American linden', 'Good', 'Alive', '40.66677776', '-73.97597938', '0101000020E610000046DB3C72767E52C0A98636F958554440')


In [27]:
### test: check the column names
with engine.connect() as connection:
    result = connection.execute(
        """
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = 'zipcodes';
        """
    )

for row in result:
    print(row)

('zipcode', 'text')
('geometry', 'USER-DEFINED')


# Part 3: Understanding the Data

In [29]:
QUERY_1 = """
SELECT incident_zip, COUNT(*) AS complaints_count
FROM complaints
WHERE
    created_date BETWEEN '2022-10-01T00:00:00.000' AND '2023-09-30T23:59:59.999'
GROUP BY incident_zip
"""

In [30]:
# table column name is the same as dataframe, not schema!!!
QUERY_2 = """
SELECT zipcode, COUNT(*) AS total_trees
FROM trees
GROUP BY zipcode
ORDER BY total_trees DESC
LIMIT 10;
"""

In [78]:
QUERY_3 = """
WITH top10_zipcodes AS (
    SELECT zipcode, COUNT(*) AS total_trees
    FROM trees
    GROUP BY zipcode
    ORDER BY total_trees DESC
    LIMIT 10
)

SELECT t.zipcode, ROUND(AVG(r.rent_amount)::numeric, 2) AS average_rent
FROM rents r
JOIN trees t ON t.zipcode = r.rent_zip::text
WHERE DATE_PART('month', r.rent_date) = 8
    AND DATE_PART('year', r.rent_date) = 2023
    AND r.rent_zip::text IN (SELECT zipcode FROM top10_zipcodes)
GROUP BY t.zipcode
ORDER BY (SELECT total_trees FROM top10_zipcodes WHERE zipcode = t.zipcode) DESC;
"""